In [1]:
from keras import backend as K
import numpy as np
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from keras.models import model_from_json
from keras.datasets import mnist
from keras.utils import np_utils
#%pylab inline
import tensorflow as tf
def resh(w):
    if len(w.shape)>2:
        i=0
        w1=1
        w2=1
        for l in w.shape:
            
            if i==0:
                w1=l
                i=1
            else:
                w2=w2*l
        
        print w2
        return np.reshape(w,(w1,w2))
    else:
        return w

im=pd.read_csv('image.csv')
image0=im.values

Using TensorFlow backend.


In [2]:
# load json and create model
json_file = open('modelL5.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
we=loaded_model.load_weights("modelL5.h5")
print("Loaded model from disk")

Loaded model from disk


In [3]:
i=0
for w in loaded_model.get_weights():
    
    if i==0:
        C1=w
    if i==1:
        C1b=w

    
    if i==2:
        C2=w
    if i==3:
        C2b=w

    
    if i==4:
        C3=w
    if i==5:
        C3b=w


    
    if i==6:
        C4=w
    if i==7:
        C4b=w
    i=i+1

print C1.shape,C1b.shape    
print C2.shape,C2b.shape    
print C3.shape,C3b.shape
print C4.shape,C4b.shape

(32, 1, 3, 3) (32,)
(32, 32, 3, 3) (32,)
(4608, 128) (128,)
(128, 10) (10,)


In [9]:
im=pd.read_csv('image.csv')
#image0=X_test[46,0,:,:]
print 'image shape:',image0.shape
image1 = tf.constant(image0,dtype='float32', shape=[1,1,28,28])

print 'image1 shape',image1
# 1st conv layer
print 'Layer I:',C1.shape, ', Layer Ib :  ',C1b.shape
#-------------------------------------#
kernel1 = C1#tf.transpose(C1, (0,1, 2,3))
print 'Transposed kernel Layer  I:',kernel1.shape
Result1=K.get_value(K.conv2d(image1 , kernel1,strides=(1, 1), border_mode='valid',dim_ordering='th'))
print 'Result of conv without C1b :',Result1.shape
Resultb1 = np.empty((1, 32, 26, 26))
for i in range(32):
    Resultb1[0,i,:,:]=Result1[0,i,:,:]+C1b[i]
#Resultb1=Result1[0,:,]+C1b

Resultb1=np.array(Resultb1)
print 'Result of conv with C1b :',Resultb1.shape
A1_res=np.array(K.get_value(K.relu(Resultb1)),)
print 'Result of conv with C1b + Relu:',A1_res.shape


image shape: (28, 28)
image1 shape Tensor("Const_9:0", shape=(1, 1, 28, 28), dtype=float32)
Layer I: (32, 1, 3, 3) , Layer Ib :   (32,)
Transposed kernel Layer  I: (32, 1, 3, 3)
Result of conv without C1b : (1, 32, 26, 26)
Result of conv with C1b : (1, 32, 26, 26)
Result of conv with C1b + Relu: (1, 32, 26, 26)


In [10]:
(pd.DataFrame(resh(Resultb1))).to_csv('file_c_1.csv',index=False)
(pd.DataFrame(resh(A1_res))).to_csv('file_c_1_relu.csv',index=False)

21632
21632


In [11]:
#-------------------------------------# Layer 2
kernel2 =C2# tf.transpose(C2, (0, 1, 2, 3))

print 'kernel2', kernel2.shape
image2 = tf.constant(A1_res[:,:,:,:],dtype='float32', shape=[1,32,26,26])
b0=np.array(C2b)
Result2=K.get_value(K.conv2d(image2 , kernel2, dim_ordering='th'))

Resultb2 = np.empty((1, 32, 24, 24))
for i in range(32):
    Resultb2[:,i,:,:]=Result2[:,i,:,:]+C2b[i]


#Resultb=Result+b0
Resultb2=np.array(Resultb2)

A2_res=np.array(K.get_value(K.relu(Resultb2)))
print A2_res.shape

kernel2 (32, 32, 3, 3)
(1, 32, 24, 24)


In [12]:
(pd.DataFrame(resh(Resultb2))).to_csv('file_c_2.csv',index=False)
(pd.DataFrame(resh(A2_res))).to_csv('file_c_2_relu.csv',index=False)

18432
18432


In [13]:


 
#-------------------------------------# MAxPool

A2R = tf.constant(A2_res,dtype='float32', shape=[1,32,24,24])
AP=K.get_value(K.pool2d(A2R, pool_size=(2,2),strides=(2,2),dim_ordering='th'))


(pd.DataFrame(resh(AP))).to_csv('file_pool.csv',index=False)

4608


In [14]:

#--------------------------------------# Flatten
KF=K.get_value(K.flatten(AP))

(pd.DataFrame(resh(KF))).to_csv('file_flat.csv',index=False)

In [15]:
print  C3.shape,KF.shape
#--------------------------------------# Dense 1

DE=(C3.T).dot(KF)+C3b

print ((C3.T).dot(KF)).shape , C3b.shape
print DE.shape

(pd.DataFrame(resh(DE))).to_csv('file_dense1.csv',index=False)

(4608, 128) (4608,)
(128,) (128,)
(128,)


In [16]:


#--------------------------------------# Dense 1 Relu
DA2_res=np.array(K.get_value(K.relu(DE)))
(pd.DataFrame(resh(DA2_res))).to_csv('file_dense1_relu.csv',index=False)

In [17]:

#--------------------------------------# Dense 2 

WDE=(C4.T).dot(DA2_res)+C4b

print WDE.shape
(pd.DataFrame(resh(WDE))).to_csv('file_dense2.csv',index=False)

(10,)


In [5]:

#--------------------------------------# softmax
a = tf.constant(np.array([WDE]))
sess=tf.Session()
print WDE
print sess.run(tf.nn.softmax(a))

kernel2 (32, 32, 3, 3)
(1, 32, 24, 24)
(4608, 128) (4608,)
(128,) (128,)
(128,)
(10,)
[ 4424.68164062 -1962.20532227  -181.99525452 -2315.38476562 -2688.57861328
 -2529.76879883  -708.00384521 -1351.1541748  -1444.06677246 -1301.04553223]
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [18]:
sess.run(tf.nn.softmax(a))[0]

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

In [6]:
#image0=X_train[46,0,:,:]
print 'image shape:',image0.shape
image1 = tf.constant(image0,dtype='float32', shape=[1,1,28,28])

ima=K.get_value(image1)
ima.shape
#image1.shape
print loaded_model.predict_classes(ima)[0]
print loaded_model.predict_proba(ima)[0]
loaded_model.summary()


image shape: (28, 28)
1/1 [==============================] - 0s
0
1/1 [==============================] - 0s
[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_8 (Convolution2D)  (None, 32, 26, 26)    320         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_13 (Activation)       (None, 32, 26, 26)    0           convolution2d_8[0][0]            
____________________________________________________________________________________________________
convolution2d_9 (Convolution2D)  (None, 32, 24, 24)    9248        activation_13[0][0]              
____________________________________________________________________________________________________
activation_14 (Activation)       (None, 32